In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser

In [6]:
#load faiss vector store from local file
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.load_local("faiss_YT_index", embeddings, allow_dangerous_deserialization=True)



In [7]:
vector_store.get_by_ids(["5a9d2117-0453-4a48-9bdc-06773b988cca"])

[Document(id='5a9d2117-0453-4a48-9bdc-06773b988cca', metadata={}, page_content='demas establish to support this podcast please check out our sponsors in the description and now let me leave you with some words from edskar dykstra computer science is no more about computers than astronomy is about telescopes thank you for listening and hope to see you next time')]

In [8]:
def format_docs(retrieved_docs):
    contect_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
    return contect_text

In [9]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})
parallel_chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever | RunnableLambda(format_docs)
    }
)
parallel_chain.invoke("who is the author of the video?")

{'question': 'who is the author of the video?',
 'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai p

In [15]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)
parser = StrOutputParser()
prompt = PromptTemplate(
    template="You are a helpful assistant. " \
              "Answer ONLY from the provided transcript context." \
              "If the contect is insufficient, just say you don'f know." \
              "\n\n {context}" \
              "\n\nQuestion: {question}",
    input_variables=["context", "question"]
)

main_chain = parallel_chain | prompt | model | parser

result = main_chain.invoke("was the topic of nuclear fusion mentioned in the video? If yes, can you provide a summary of the topic discussion?")

print(result)

Yes, the topic of nuclear fusion was mentioned in the video. The discussion focused on the collaboration with EPFL in Switzerland, where a test reactor was used to explore bottleneck problems in fusion. The speaker described how they applied AI methods to address these challenges, particularly in controlling and holding plasma in specific shapes for extended periods, which is crucial for energy production. They mentioned a published paper on this work and expressed interest in collaborating with fusion startups to tackle further problems in the field. The speaker also highlighted the broader implications of using AI in energy and climate challenges, indicating that fusion is one area where AI could be transformative.
